# SC4020

# references 
# https://github.com/mie-lab/trackintel 
- for import 

# https://trackintel.readthedocs.io/en/latest/index.html


In [1]:
# pip install trackintel pandas prefixspan

In [1]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
import trackintel as ti
# import torch
import matplotlib.pyplot as plt
from datetime import timedelta
import math

# Data preprocessing

### Load data

In [2]:
'''df_hiroshima = pd.read_csv('hiroshima_challengedata.csv')
reference_lat = 34.3853
reference_lon = 132.4553
df_hiroshima.head()

df_sapporo = pd.read_csv('sapporo_challengedata.csv')
reference_lat = 43.0618
reference_lon = 141.3545
df_sapporo.head()

df_kotae = pd.read_csv('task1_dataset_kotae.csv')
reference_lat = 33.6925
reference_lon = 130.7127
df_kotae.head()

#reference_lat = 32.8032
#reference_lon = 130.7079'''

## modify to choose dataset/frame
#df = df_kotae.copy()
#df = df_sapporo.copy()
#df = df_hiroshima.copy()

## for csv generation filename 
#"sapporo_freq_subseq.csv"
#"kotae_freq_subseq.csv"
#"hiroshima_freq_subseq.csv"

df_kumamoto = pd.read_csv('kumamoto_challengedata.csv')
df = df_kumamoto.copy()
csv_filename = "kumamoto_freq_subseq.csv"

reference_lat = 0
reference_lon = 0
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8418135 entries, 0 to 8418134
Data columns (total 5 columns):
 #   Column  Dtype
---  ------  -----
 0   uid     int64
 1   d       int64
 2   t       int64
 3   x       int64
 4   y       int64
dtypes: int64(5)
memory usage: 321.1 MB


from google.colab import drive
drive.mount('/content/drive')

In [3]:
# TODO: remove - this is to speed up processing for initial testing
# df = df.iloc[:100000]
#first_1000_uids = df['uid'].drop_duplicates().head(1000)

# Filter the DataFrame based on the first 1000 unique user IDs
#df = df[df['uid'].isin(first_1000_uids)]

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column  Non-Null Count   Dtype
---  ------  --------------   -----
 0   uid     100000 non-null  int64
 1   d       100000 non-null  int64
 2   t       100000 non-null  int64
 3   x       100000 non-null  int64
 4   y       100000 non-null  int64
dtypes: int64(5)
memory usage: 3.8 MB


### Data processing

### Reduce datapoints (from tips in notes -- Scope of Data Analysis)
Decrease processing time as dataset is large. Take only user's first 30 days.

In [4]:
def process_data(df):
    # drop invalid
    print("Processing Data")
    df = df[(df['x'] != -999) & (df['y'] != -999)]
    # up to 30 days(total 75), mainly to reduce dataset size to reduce processing time (after 65 days dataset not complete too)

    df = df[df['d'] <= 30]

    # Users with too few interactions (infrequent users). min 10
    # reduce first for faster processing speed
    user_counts = df['uid'].value_counts()
    frequent_users = user_counts[user_counts > 10].index
    df = df[df['uid'].isin(frequent_users)]

    df = df.reset_index(drop=True)

    ### Data Transformation
    # Transform data so it can fit into trackintel processing formats

    # Adjust 'd' to a date for reference
    start_date = pd.to_datetime('2024-01-01')
    df['date'] = start_date + pd.to_timedelta(df['d'], unit='D')  # Convert 'd' to a date

    # Convert 't' to hours and minutes, and create a formatted datetime column directly
    df['tracked_at'] = pd.to_datetime(df['date'].dt.strftime('%Y-%m-%d') + ' ' +
                                      (df['t'] // 2).astype(str).str.zfill(2) + ':' +
                                      ((df['t'] % 2) * 30).astype(str).str.zfill(2),
                                      format='%Y-%m-%d %H:%M')

    # Convert to reference with latitude and longitude to aid with haversine calculations
    grid_unit_to_degrees = 0.0045
    df['longitude'] = reference_lon + df['x'] * grid_unit_to_degrees
    df['latitude'] = reference_lat + df['y'] * grid_unit_to_degrees

    # Create geometry column 
    df['geometry'] = df.apply(lambda row: Point(row['longitude'], row['latitude']), axis=1)

    # Convert to a GeoDataFrame
    gdf = gpd.GeoDataFrame(df, geometry='geometry', crs="EPSG:4326")# CRS 4326 
    return gdf

### Data processing (continued, require new columns from transformations to filter)
- Remove short sequences with insufficient data points for pattern analysis.(below 7 days)
- Records that fall outside the specified time window (recent activity). (minimum of 3 records in 7 rolling days)
- This results in a dataset that’s cleaner and better suited for identifying meaningful location patterns.

In [5]:
''' Fucntion to get valid sequences where min records in certain number of days must be met'''
def get_valid_sequences(df, days=7, min_records=3):
    print("Getting/Filtering valid sequences")
    # Calculate days since the first record for each user
    df = df.sort_values(["uid", "tracked_at"])
    df["days_since_start"] = df.groupby("uid")["tracked_at"].transform(lambda x: (x - x.min()).dt.days)
    
    # Filter out sequences with less than the minimum days required
    df = df[df["days_since_start"] >= days]
    
    # Use rolling window to check the count of records in the past 'days' for each user
    valid_sequences = []
    for uid, user_data in df.groupby("uid"):
        user_data["record_count"] = user_data["days_since_start"].rolling(window=days).count().shift(1)  # Shift by 1 to exclude current row
        valid_sequences.append(user_data[user_data["record_count"] >= min_records])
    
    # Concatenate all valid user data
    valid_sequences = pd.concat(valid_sequences)
    return valid_sequences.reset_index(drop=True)



In [6]:
### Remove unecessary columns and format datatypes -- faster processing
def reduce_gdf(gdf):
    print("Removing unecessary columns GDF")
    gdf_processed = gdf[["uid", "tracked_at", "geometry"]]

    # Convert the 'datetime_column' back to string
    gdf_processed.loc[:, 'tracked_at'] = gdf_processed['tracked_at'].astype(str)

    return gdf_processed

# TrackIntel 
Here we use Trackintel to process our data to eventually obtain triplegs

## Get positionfixes

## Get staypoints

### Important parameters guide for generating staypoints

dist_threshold (float, default=100)
Description: (in meters if using 'haversine') Distance a user can move between two consecutive position fixes to be considered part of the same staypoint.

time_threshold (float, default=5.0)
Description: The time threshold (in minutes) that determines how long the user must stay in the same area for it to be considered a staypoint.

gap_threshold (float, default=15.0)
Description: The time gap threshold (in minutes) to determine whether a temporal gap exists between two consecutive position fixes. If the time gap exceeds this threshold, the consecutive fixes are treated as separate stays.

include_last (boolean, default=False)
Description: Whether to include the last staypoint in the data. 
Usage: Set to True if you want to include the last staypoint, even if the user hasn't stepped out of the location by the end of the data.

-------------------------------------------------

Our data is split in 30 minutes gaps.

## Generate Triplegs
gap_threshold value is set to 91

In [7]:
def generate_pfs_sp_tpls(gdf_processed):
    # Read position fixes from GeoDataFrame, CRS None - operations on raw coordinates values
    print("Reading PositionFixies")
    df_pfs = ti.io.read_positionfixes_gpd(gdf_processed, tracked_at='tracked_at', user_id='uid', geom_col="geometry", crs=None, tz=None, mapper=None)

    # Generate staypoints 
    ''' 
    dist_threshold : 1000m movement area allowed within staypoint  
    time_threshold : 60 min to be considered staypoint(2 gps tracks)
    gap_threshold : 240min. Consecutive pfs with temporal gaps larger than ‘gap_threshold’ will be excluded from staypoints generation. Assume user left and returned in between
    '''
    print("Generating Staypoints")
    df_pfs, df_sp = df_pfs.generate_staypoints(method='sliding', distance_metric='haversine', 
                                                                    dist_threshold=1000, time_threshold=60, gap_threshold=240, include_last = False, 
                                                                    print_progress = True, n_jobs = -1)

    df_pfs = ti.io.read_positionfixes_gpd(df_pfs, tracked_at='tracked_at', user_id='uid', geom_col="geometry", crs=None, tz=None, mapper=None)

    #Generate triplegs
    print("Generating Triplegs")
    df_pfs, df_tpls = ti.preprocessing.generate_triplegs(df_pfs, staypoints=df_sp, method='between_staypoints', gap_threshold=90)

    df_pfs = ti.io.read_positionfixes_gpd(df_pfs, tracked_at='tracked_at', user_id='uid', geom_col="geometry", crs=None, tz=None, mapper=None)
    df_tpls = df_tpls.reset_index(drop=True)
    return df_pfs, df_sp, df_tpls

In [8]:
# Generate geodataframe ready to feed into trackintel
gdf = process_data(df)
gdf = get_valid_sequences(gdf)
gdf_processed = reduce_gdf(gdf)
print(gdf_processed.info())

# generate pfs, sp and tpls
df_pfs, df_sp, df_tpls = generate_pfs_sp_tpls(gdf_processed)

Processing Data


D:\Anaconda\envs\loc-pred\lib\site-packages\trackintel\io\from_geopandas.py:431: UserWarning: Assuming UTC timezone for column tracked_at
  warnings.warn(f"Assuming UTC timezone for column {col_name}")


Getting/Filtering valid sequences
Removing unecessary columns GDF
<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 31775 entries, 0 to 31774
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   uid         31775 non-null  int64         
 1   tracked_at  31775 non-null  datetime64[ns]
 2   geometry    31775 non-null  geometry      
dtypes: datetime64[ns](1), geometry(1), int64(1)
memory usage: 744.9 KB
None
Reading PositionFixies
Generating Staypoints


100%|██████████| 65/65 [00:05<00:00, 11.92it/s] 


Generating Triplegs


D:\Anaconda\envs\loc-pred\lib\site-packages\trackintel\preprocessing\positionfixes.py:361: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  pfs["tripleg_id"] = pfs["tripleg_id"].ffill()
D:\Anaconda\envs\loc-pred\lib\site-packages\trackintel\preprocessing\positionfixes.py:508: UserWarning: The positionfixes with ids [  783   784   785   786   851   852   992   993  1013  1014  1069  1070
  1186  1187  1188  1486  1487  1598  1599  1600  1995  1996  2082  2083
  2084  2093  2094  2852  2853  2854  3158  3159  3636  3637  3737  3738
  4688  4689  5355  5356  5357  5358  5359  5360  5361  5362  5363  5364
  5365  5366  5367  6444  6445  6446  6530  6531  7349  7350  7653  7654
  8301  8302  8303  8467  8468  8537  8538  8699  8700 13920 13921 14015
 14016 14076 14077 14078 140

## Processing Triplegs data
- Map tripleg coordinates to zones - zoning(group) coordinates together
 - this is done as previous mining with coordinates didnt yield enough results, so more generalised space for points
 


In [9]:
# Function to scale and round coordinates of a point to whole numbers
def convert_point_coordinates(point, scale_factor):
    x = round(point.x * scale_factor)  # Rounding to the nearest whole number
    y = round(point.y * scale_factor)  # Rounding to the nearest whole number
    return (x,y)

# Function to map the geometry of a LINESTRING to scaled (x, y) coordinates
def map_linestring_to_coordinates(linestring, scale_factor):
    coordinates = []
    for point in linestring.coords:
        converted_point = convert_point_coordinates(Point(point), scale_factor)
        coordinates.append(converted_point)
    return coordinates

# Function to convert triplegs in a GeoDataFrame to scaled coordinates
def convert_triplegs_to_coordinates(gdf, scale_factor=1/0.0045):
    gdf["coords"] = gdf["geom"].apply(
        lambda geom: map_linestring_to_coordinates(geom, scale_factor)
    )
    return gdf
    
## NOT IN USE
# Zoning
def process_coordinates(coords):
    grouped_coords = []
    for x, y in coords:
        x_scaled = x / 2
        y_scaled = y / 2
        
        x_floor = math.floor(x_scaled)
        y_floor = math.floor(y_scaled)
        
        x_grouped = x_floor * 2
        y_grouped = y_floor * 2
        
        grouped_coords.append((x_grouped, y_grouped))
    
    return grouped_coords

In [10]:
df_tpls_originalgrid = convert_triplegs_to_coordinates(df_tpls)

# Zone them 
df_tpls_originalgrid['processed_coords'] = df_tpls_originalgrid['coords'].apply(process_coordinates)

# Data Analysis : GSP Algorithm

### Pre-process data to retrieve sequences

In [11]:
# Generate sequence
sequence = df_tpls_originalgrid["coords"].tolist()

### GSP Algorithm implementation

Reference : https://github.com/jacksonpradolima/gsp-py/blob/master/gsppy/gsp.py

Core Steps: <br>
Candidate Generation (using prefix pruning). <br>
Support Counting (count how often subsequences appear in the dataset).<br>
Iterative Mining (search for subsequences of increasing lengths).<br>



In [12]:
import customgsp
import time

start_time = time.time()

# param is minsup. finds common routes people tend to take at in a trip.
min_sup = round(len(sequence) * 0.001)
print(f"Min Sup: {min_sup}")
result = customgsp.CUSTOMGSP(sequence).search(min_sup)

end_time = time.time()
# Calculate and print the total time taken
print(f"Start time: {start_time}")
print(f"End time: {end_time}")
print(f"Total time taken: {end_time - start_time} seconds")

Min Sup: 3
Start time: 1732188142.3796892
End time: 1732189179.2617865
Total time taken: 1036.8820972442627 seconds


In [13]:
for idx, dictionary in enumerate(result):
        print(f"Length {idx + 1}:")
        for key, value in dictionary.items():
            # Check if key is a tuple and join elements if needed
            key_str = ', '.join(map(str, key))
            print(f"Zone {key_str}: {value}")
        print("\n" + "-"*50 + "\n")

Length 1:
Zone (143, 98): 19
Zone (144, 100): 4
Zone (116, 134): 13
Zone (144, 97): 15
Zone (153, 69): 3
Zone (134, 104): 6
Zone (133, 124): 10
Zone (140, 110): 7
Zone (118, 97): 3
Zone (121, 99): 14
Zone (136, 121): 7
Zone (136, 122): 14
Zone (126, 127): 6
Zone (136, 115): 3
Zone (141, 107): 6
Zone (126, 100): 8
Zone (120, 99): 22
Zone (118, 98): 3
Zone (117, 94): 21
Zone (119, 99): 12
Zone (123, 99): 3
Zone (145, 98): 3
Zone (162, 99): 3
Zone (148, 97): 3
Zone (117, 96): 18
Zone (151, 98): 3
Zone (163, 98): 3
Zone (133, 103): 3
Zone (122, 99): 18
Zone (144, 98): 12
Zone (137, 123): 3
Zone (130, 125): 5
Zone (131, 78): 5
Zone (126, 71): 17
Zone (126, 73): 8
Zone (142, 98): 6
Zone (136, 82): 17
Zone (126, 72): 7
Zone (126, 68): 6
Zone (129, 71): 4
Zone (144, 99): 8
Zone (143, 97): 6
Zone (129, 126): 3
Zone (117, 135): 4
Zone (132, 125): 5
Zone (140, 105): 4
Zone (163, 99): 3
Zone (131, 125): 5
Zone (119, 107): 10
Zone (109, 101): 3
Zone (101, 103): 15
Zone (101, 104): 14
Zone (115, 95)

In [ ]:
import csv
# Extract only keys
filtered_result = [
    {k: v for k, v in d.items() if len(k) > 1}
    for d in result
]
keys_list = []
for record in filtered_result:
    keys_list.extend(record.keys())

# Write to the CSV
with open(csv_filename, mode="w", newline="") as file:
    writer = csv.writer(file)
    writer.writerow(["Pattern"])  # Write header
    for key in keys_list:
        writer.writerow([key])

print(f"Keys saved to {csv_filename}")

### Appendix/Other (obsolete) functions

In [14]:
from prefixspan import PrefixSpan
ps = PrefixSpan(sequence)
min_sup = round(len(sequence) * 0.0001)
print(f"Min Sup: {min_sup}")
freq_subseq = ps.frequent(min_sup) 

Min Sup: 23


In [17]:
import csv

# Extract the lists
extracted_lists = [item[1] for item in freq_subseq if len(item[1]) > 1]

# Convert each row into a single-column format
formatted_data = [["({})".format(", ".join(map(str, row)))] for row in extracted_lists]

# Write to a CSV file
with open("output.csv", mode="w", newline="") as file:
    writer = csv.writer(file)
    writer.writerow(["Patterns"])  # Header
    writer.writerows(formatted_data)

print("Data saved to output.csv")


Data saved to output.csv


In [ ]:
# Plot Staypoints
df_sp['x'] = df_sp['geometry'].apply(lambda point: point.x)
df_sp['y'] = df_sp['geometry'].apply(lambda point: point.y)

plt.figure(figsize=(8, 6))
plt.scatter(df_sp['x'], df_sp['y'], c='blue', label='Staypoints')
plt.title('Staypoints Locations')
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.legend()
plt.grid()
plt.show()